In [171]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [172]:
mydir = '/Users/supriyajadhav/Documents/MS422-Practical-Machine-Learning/Project422/'
processed_df = pd.read_csv(mydir + 'preprocessed_train_data.csv')
test_df = pd.read_csv(mydir + 'preprocessed_test_data.csv')

/var/folders/r1/dsylm7cn0vg3015hchpddz9m0000gn/T/ipykernel_72520/3545090819.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  processed_df = pd.read_csv(mydir + 'preprocessed_train_data.csv')


In [173]:
# Convert 'date' to datetime format
processed_df['date'] = pd.to_datetime(processed_df['date'])

# Extract day,month and year
processed_df['day'] = processed_df['date'].dt.day

processed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3008280 entries, 0 to 3008279
Data columns (total 20 columns):
 #   Column         Dtype         
---  ------         -----         
 0   Unnamed: 0     int64         
 1   id             int64         
 2   date           datetime64[ns]
 3   store_nbr      int64         
 4   family         object        
 5   sales          float64       
 6   onpromotion    int64         
 7   city           object        
 8   state          object        
 9   type           object        
 10  cluster        int64         
 11  holiday_type   object        
 12  IS_HOLIDAY     float64       
 13  oil_price      float64       
 14  month          int64         
 15  year           int64         
 16  week_of_day    int64         
 17  week_of_month  int64         
 18  week_number    int64         
 19  day            int64         
dtypes: datetime64[ns](1), float64(3), int64(11), object(5)
memory usage: 459.0+ MB


In [174]:
# Filter processed_df to only include rows where store_nbr is in (1, 2, 3, 4, 5)
processed_df = processed_df[processed_df['store_nbr'].isin([51, 52, 53, 54])]

# Filter test_df to only include rows where store_nbr is in (1, 2, 3, 4, 5)]
test_df = test_df[test_df['store_nbr'].isin([51, 52, 53, 54])]

result_file = 's_sales_prediction_51_54.csv'

In [175]:
test_df['date'] = pd.to_datetime(test_df['date'])
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2112 entries, 1518 to 28379
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Unnamed: 0     2112 non-null   int64         
 1   id             2112 non-null   int64         
 2   date           2112 non-null   datetime64[ns]
 3   store_nbr      2112 non-null   int64         
 4   family         2112 non-null   object        
 5   onpromotion    2112 non-null   int64         
 6   city           2112 non-null   object        
 7   state          2112 non-null   object        
 8   type           2112 non-null   object        
 9   cluster        2112 non-null   int64         
 10  IS_HOLIDAY     2112 non-null   float64       
 11  day            2112 non-null   int64         
 12  month          2112 non-null   int64         
 13  year           2112 non-null   int64         
 14  week_of_day    2112 non-null   int64         
 15  week_of_month  21

In [176]:
unique_store_family_df = processed_df[['store_nbr', 'family']].drop_duplicates()

# Reset index if you want a clean DataFrame
unique_store_family_df = unique_store_family_df.reset_index(drop=True)

# Add a column for RMSLE values
unique_store_family_df['val_RMSLE'] = None

# Display the resulting DataFrame
print(len(unique_store_family_df))
print(unique_store_family_df.head())


132
   store_nbr      family val_RMSLE
0         51  AUTOMOTIVE      None
1         51   BABY CARE      None
2         51      BEAUTY      None
3         51   BEVERAGES      None
4         51       BOOKS      None


In [177]:
missing_data = processed_df.isnull().sum()
print(missing_data)

Unnamed: 0            0
id                    0
date                  0
store_nbr             0
family                0
sales                 0
onpromotion           0
city                  0
state                 0
type                  0
cluster               0
holiday_type     203544
IS_HOLIDAY            0
oil_price         63921
month                 0
year                  0
week_of_day           0
week_of_month         0
week_number           0
day                   0
dtype: int64


In [178]:
processed_df = processed_df.drop(['holiday_type', 'oil_price'], axis = 1)

In [179]:
# Group train data
# Step 1: Group by 'store_nbr' and 'family'
grouped = processed_df.groupby(['store_nbr', 'family'])

# Step 2: Sort each group by 'date' in ascending order
sorted_df = grouped.apply(lambda x: x.sort_values('date')).reset_index(drop=True)

# Optional: If you want to create a dictionary of DataFrames, one for each group
grouped_dict = {f"processed_{store}_{family}": group.sort_values('date') 
                for (store, family), group in grouped}

In [180]:
# group test data
# Step 1: Group by 'store_nbr' and 'family'
grouped_test = test_df.groupby(['store_nbr', 'family'])

# Step 2: Sort each group by 'date' in ascending order
sorted_test_df = grouped_test.apply(lambda x: x.sort_values('date')).reset_index(drop=True)

# Optional: If you want to create a dictionary of DataFrames, one for each group
grouped_test_dict = {f"test_{store}_{family}": group.sort_values('date') 
                for (store, family), group in grouped_test}

/var/folders/r1/dsylm7cn0vg3015hchpddz9m0000gn/T/ipykernel_72520/3328157936.py:6: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  sorted_test_df = grouped_test.apply(lambda x: x.sort_values('date')).reset_index(drop=True)


In [181]:
import tensorflow as tf

# defien RMSLE metric
def rmsle(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float64)
    y_pred = tf.cast(y_pred, tf.float64)
    return tf.sqrt(tf.reduce_mean(tf.square(tf.math.log(y_pred + 1) - tf.math.log(y_true + 1))))


In [182]:
# placeholder test data predictions
predictions_list=[]

In [183]:
# Loop through each unique combination
for _, row in unique_store_family_df.iterrows():
    store = row['store_nbr']
    family = row['family']

    # Get the corresponding train and test subset DataFrame
    subset_df = grouped_dict[f'processed_{store}_{family}']
    test_subset_df = grouped_test_dict[f'test_{store}_{family}']

    # combine test and train data for scaling
    tt_df = pd.concat([subset_df,test_subset_df])

    # Create cyclical features
    tt_df['month_sin'] = np.sin(2 * np.pi * tt_df['month'] / 12)
    tt_df['month_cos'] = np.cos(2 * np.pi * tt_df['month'] / 12)
    tt_df['weekday_sin'] = np.sin(2 * np.pi * tt_df['week_of_day'] / 7)
    tt_df['weekday_cos'] = np.cos(2 * np.pi * tt_df['week_of_day'] / 7)

    # Drop rows with NaN values if needed  (excpet for sales as test data does nkt have sales data)
    tt_df = tt_df.dropna(subset=[col for col in tt_df.columns if col != 'sales'])

    # drop unnecessary columns
    tt_df = tt_df.drop(['Unnamed: 0', 'date', 'id', 'city', 'state', 'store_nbr', 'family', 'type' ], axis = 1)

    # Convert data to float64
    tt_df = tt_df.astype('float64')

    # Selecting features and target
    features = ['onpromotion', 'cluster', 'IS_HOLIDAY', 'month', 'year',
       'week_of_day', 'week_of_month', 'week_number', 'day', 'month_sin',
       'month_cos', 'weekday_sin', 'weekday_cos']
    
    target = 'sales'
    
    # Feature scaling
    feature_scaler = MinMaxScaler(feature_range=(0, 1))
    target_scaler = MinMaxScaler(feature_range=(0, 1))

    # Get the remaining records for the training set
    feature_df = pd.DataFrame(feature_scaler.fit_transform(tt_df[features]))
    target_df = pd.DataFrame(target_scaler.fit_transform(tt_df[target].dropna().values.reshape(-1,1)))

    # split train and test
    X = feature_df.iloc[:-16]
    y = target_df

    sep_test_df = feature_df.iloc[-16:]

    # Splitting data into training and testing sets
    X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = train_test_split(X, y, test_size=0.2, shuffle=False)

    # LSTM Model
    print(f'\n Training for store {store} and family {family} \n')  

    # Reshape data to fit LSTM input shape (num_samples, timesteps, num_features)
    timesteps = 1  # You need to define this based on your data and model requirements
    X_train_reshaped = X_train_scaled.values.reshape((X_train_scaled.values.shape[0], timesteps, X_train_scaled.values.shape[1]))
    X_test_reshaped = X_test_scaled.values.reshape((X_test_scaled.values.shape[0], timesteps, X_test_scaled.values.shape[1]))

    # Building the LSTM model
    lstm_model = Sequential()
    lstm_model.add(LSTM(units=60, return_sequences=True, input_shape=(timesteps, X_train_reshaped.shape[2])))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(LSTM(units=60, return_sequences=False))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units=1))

    lstm_model.compile(optimizer='adam', loss=rmsle)
    #lstm_model.summary()

    # Define early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Training the LSTM model
    lstm_model.fit(X_train_reshaped, y_train_scaled, epochs=50, batch_size=32, 
               validation_data=(X_test_reshaped, y_test_scaled),
               callbacks=[early_stopping])

    ## Predictions#####################################
    lstm_predictions = lstm_model.predict(X_test_reshaped)

    # Ensure both predictions and actual values are of type float64
    y_test_scaled = y_test_scaled.values.reshape(-1, 1).astype('float64')
    lstm_predictions_scaled = lstm_predictions.astype('float64')

    # Calculate RMSLE
    lstm_rmsle_val = rmsle(y_test_scaled, lstm_predictions_scaled)

    unique_store_family_df.loc[
        (unique_store_family_df['store_nbr'] == store) &
        (unique_store_family_df['family'] == family),
        'val_RMSLE'
        ] = lstm_rmsle_val.numpy()

    print(f"LSTM RMSLE for store: {store} and family: {family} is : {lstm_rmsle_val.numpy()}")

    # perform data pre processing on kaggel test.csv#######################################

    # resahaping for LSTM
    test_reshaped = sep_test_df.values.reshape((sep_test_df.shape[0], 
                                                timesteps, 
                                                sep_test_df.shape[1]))

    # Make predictions using the LSTM model
    lstm_predictions_test_normalised = lstm_model.predict(test_reshaped)

    # inverse transform
    lstm_predictions_test_normalised_reshaped = lstm_predictions_test_normalised.astype('float64')
    lstm_predictions_test_actual = target_scaler.inverse_transform(lstm_predictions_test_normalised_reshaped).astype('float64')

    predictions_df = pd.DataFrame({
        'id': test_subset_df['id'],
        'store_nbr': store,
        'family': family,
        'date': test_subset_df['date'], 
        'predictions': lstm_predictions_test_actual.flatten(), # Ensure predictions is 1D
        'val_rmsle': lstm_rmsle_val.numpy()  
    })

    # Round predictions to 4 decimal places and replace negative values with 0
    predictions_df['predictions'] = predictions_df['predictions'].round(4)
    predictions_df['predictions'] = predictions_df['predictions'].apply(lambda x: max(0, x))

    predictions_list.append(predictions_df)



 Training for store 51 and family AUTOMOTIVE 

Epoch 1/50


/Users/supriyajadhav/anaconda3/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


43/43 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: nan - val_loss: nan
Epoch 2/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: nan - val_loss: nan
Epoch 3/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: nan - val_loss: nan
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
LSTM RMSLE for store: 51 and family: AUTOMOTIVE is : nan
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step

 Training for store 51 and family BABY CARE 

Epoch 1/50


/Users/supriyajadhav/anaconda3/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


43/43 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.0216 - val_loss: 0.0035
Epoch 2/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0124 - val_loss: 0.0014
Epoch 3/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0087 - val_loss: 0.0014
Epoch 4/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0068 - val_loss: 8.4658e-04
Epoch 5/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0053 - val_loss: 0.0011
Epoch 6/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0040 - val_loss: 6.9119e-04
Epoch 7/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0034 - val_loss: 0.0011
Epoch 8/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0026 - val_loss: 3.9818e-04
Epoch 9/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0020 - val_loss: 2.7495e-04
Epoch 10/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0016 - val_loss: 1.6248e-04
Epoch 11/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0012 - val_loss: 1.0559e-04
Epoch 12/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.

...

43/43 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: nan - val_loss: nan
Epoch 2/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: nan - val_loss: nan
Epoch 3/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: nan - val_loss: nan
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
LSTM RMSLE for store: 54 and family: PRODUCE is : nan
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step

 Training for store 54 and family SCHOOL AND OFFICE SUPPLIES 

Epoch 1/50


/Users/supriyajadhav/anaconda3/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


43/43 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.2771 - val_loss: 0.3359
Epoch 2/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1885 - val_loss: 0.2991
Epoch 3/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1697 - val_loss: 0.2529
Epoch 4/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1605 - val_loss: 0.2391
Epoch 5/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1366 - val_loss: 0.2214
Epoch 6/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1469 - val_loss: 0.2120
Epoch 7/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1372 - val_loss: 0.2041
Epoch 8/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1329 - val_loss: 0.2094
Epoch 9/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1380 - val_loss: 0.2088
Epoch 10/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1231 - val_loss: 0.2072
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
LSTM RMSLE for store: 54 and family: SCHOOL AND OFFICE SUPPLIES is : 0.31346811256091944
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step

 Tr

/Users/supriyajadhav/anaconda3/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


43/43 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 1.0456 - val_loss: 0.9241
Epoch 2/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.9408 - val_loss: 0.8880
Epoch 3/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8935 - val_loss: 0.8763
Epoch 4/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.9207 - val_loss: 0.8798
Epoch 5/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.9156 - val_loss: 0.8831
Epoch 6/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8931 - val_loss: 0.8947
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
LSTM RMSLE for store: 54 and family: SEAFOOD is : 0.8794490680594873
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


In [184]:
final_predictions_df = pd.concat(predictions_list, ignore_index=True)
final_predictions_df.to_csv(mydir + result_file)

In [188]:
import os
import glob

# Find all files starting with 'sales_prediction_' and read them into DataFrames
file_list = glob.glob(os.path.join(mydir, 's_sales_prediction_*.csv'))
final_df = pd.concat((pd.read_csv(file, usecols=['id', 'predictions', 'val_rmsle']) for file in file_list), ignore_index=True)

mean_rmsle = final_df['val_rmsle'].sum() / len(final_df)
final_df = final_df.drop(columns=['val_rmsle'])

# Sort by 'id' and rename 'predictions' to 'sales'
final_df = final_df.sort_values(by='id').reset_index(drop=True)
final_df.rename(columns={'predictions': 'sales'}, inplace=True)

# Save the final DataFrame to a CSV file
final_df.to_csv(os.path.join(mydir, 's_final_sales_predictions.csv'), index=False)

In [189]:
# Display the first few rows
print(final_df.head())
print(len(final_df))

        id   sales
0  3000888  0.0000
1  3000889  0.0000
2  3000890  0.0000
3  3000891  0.0000
4  3000892  0.0538
28512


In [190]:
print(f'The mean RMSLE for the LSTM Model over 1782 series is {mean_rmsle}')
print('Training & Inference duration: 3114 seconds')

The mean RMSLE for the LSTM Model over 1782 series is 0.34865558886068154
Training & Inference duration: 3114 seconds
